In [46]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By

In [2]:
URL_START = 'https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text='

SEARCH_OPTIONS = {"search_by_name":"&search_field=name", "search_by_name_and_description":"&search_field=name&search_field=description"}

URL_MID = '&logic=normal&pos=full_text&exp_period=all_time&exp_company_size=any&show_conditions=false&filter_exp_period=all_time&area='

URL_END = '&salary=&currency_code=RUR&only_with_salary=true&experience=doesNotMatter&employment=full&schedule=fullDay&order_by=publication_time&search_period=0&items_on_page=100&page='


In [63]:
driver = webdriver.Chrome(executable_path=r"C:\Chromedriver\chromedriver.exe")

In [2]:
def ParseHHJobs(query, search_regions_info, target_dict_for_df, search_by_name_and_description = True):
    preliminary_url = URL_START + query
    
    if (search_by_name_and_description == True):
        preliminary_url += SEARCH_OPTIONS["search_by_name_and_description"]
    else:
        preliminary_url += SEARCH_OPTIONS["search_by_name"]
    
    preliminary_url += URL_MID
    
    for region, region_data in search_regions_info.items():
        search_url = preliminary_url + str(region_data[0]) + URL_END
        
        for i in range(0, region_data[1]):
            driver.get(search_url + str(i))
            
            vacancies = driver.find_elements_by_xpath('//div[@class="vacancy-info--I4f9shQE53f9Luf5lkMw"]')
            
            vacancy_names = driver.find_elements_by_xpath('//span[@class="magritte-text___pbpft_3-0-13 magritte-text_style-primary___AQ7MW_3-0-13 magritte-text_typography-title-4-semibold___vUqki_3-0-13"]/span[@class="magritte-text___tkzIl_4-2-6"]')
            wages = driver.find_elements_by_xpath('//span[@class="magritte-text___pbpft_3-0-13 magritte-text_style-primary___AQ7MW_3-0-13 magritte-text_typography-label-1-regular___pi3R-_3-0-13"]')
            employers = driver.find_elements_by_xpath('//span[@data-qa="vacancy-serp__vacancy-employer-text"]')
            links_raw = driver.find_elements_by_xpath('//a[@class="magritte-link___b4rEM_4-2-6 magritte-link_style_neutral___iqoW0_4-2-6 magritte-link_enable-visited___Biyib_4-2-6"]')
            links = [elem.get_attribute('href') for elem in links_raw]

            wages_cleaned_up = []
            employers_cleaned_up = []

            for i in wages:
                if i.text != "":
                    wages_cleaned_up.append(i.text)

            for i in employers:
                if i.text != "":
                    employers_cleaned_up.append(i.text)

            print("Diagnosis info")
            print("Vacancies", len(vacancies))
            print("Vacancy_names_len", len(vacancy_names))
            print("Wages_list_len", len(wages_cleaned_up))
            print("Employers_list_len", len(employers_cleaned_up))
            print("Links_len", len(links))
            print("__________________________________")


            for p in range(len(vacancy_names)):
                target_dict_for_df["Vacancy_names"].append(vacancy_names[p].text)
                print("Vacancy_name: ", vacancy_names[p].text)

                try:
                    target_dict_for_df["Wages_list"].append(wages_cleaned_up[p])
                    print("Wage: ", wages_cleaned_up[p])
                except:
                    target_dict_for_df["Wages_list"].append("-")
                    print("Wage: ", "-")

                try:
                    target_dict_for_df["Employers_list"].append(employers_cleaned_up[p])
                    print("Employer: ", employers_cleaned_up[p])
                except:
                    target_dict_for_df["Employers_list"].append("-")
                    print("Employer: ", "-")

                target_dict_for_df["Regions_list"].append(region)
                target_dict_for_df["Links"].append(links[p])
                print(links[p])
                print(region)
                print("__________________________________")

In [53]:
search_region_info_HIRING = {"Волгоград": [24,2], "Ижевск": [96,2], "Казань": [88,2], "Саратов": [79,2], "Тольятти": [212,2], "Самара": [78,2], "Ярославль": [112,2]}

In [95]:
dict_for_df_HIRING = {"Vacancy_names":[], "Wages_list":[], "Search_status":[], "Regions_list":[], "Links":[]}

In [96]:
ParseHHJobs("подбор персонала", search_region_info_HIRING, dict_for_df_HIRING)

Diagnosis info
Vacancies 101
Vacancy_names_len 101
Wages_list_len 101
Employers_list_len 100
Links_len 101
__________________________________
Vacancy_name:  Управляющая/Управляющий Кафе
Wage:  50 000 – 65 000 ₽ на руки
Employer:  ИП Быкова Жанна Юрьевна
https://adsrv.hh.ru/click?b=1158458&place=35&meta=KfrW3FQoZCk6YZWt4rktC-BBxqispTRn6881-efSc0hpFc1o48JBtIxa2-nmfMYUsFtR8lTN7iX4JAK_abI7diezxura2qP-2jRT7OOZjj9gR2-fHBgnM6h1SB2uFgDZ3O7_OexxhFl181UkufO0FBhYwAhJvMAoW1_xMDAQN-4nfbIL4IRZ-sOnEnPaqA7TmUn-TtTAd_ih_PPFyrww-TR4Fu48sjoA2hBB9SNxjzXcoit4jtrLhe6btVUAaVWoUWRbIRrgifAnYi-9mb87Ec-44xKZKhoenILLsjIUnYDRmDhIz6zqpa1cmHFBj-gC62t6E5ODCKKDeczQEVYHdAfvqYFzmzof--LHA-0Kpvw6A0oRI8VQwaNCxKr_eN3UM_En8-fAdEWE6puPfU9OP43juo1jvHZv1fuT7g4xS4G8Jwib6PjyqNDKu35ViKpl18CPecCbOnU-C9f9xypYGFxAj9jK8IrQIuyD6SnY3kZmOkmxR-wdFtAsDLty72zS8pj1cSpBKRscrQ1JsQYrUQ%3D%3D&clickType=link_to_vacancy
Волгоград
__________________________________
Vacancy_name:  Менеджер по подбору персонала
Wage:  50 000 – 60 000 ₽ на руки
Employ

In [88]:
len(dict_for_df_HIRING[])

4

In [51]:
df.to_csv('out.csv')

In [34]:
def test():
    return x


In [ ]:
def ParseCityAccountantVacanciesTEST(number_of_pages, region_number):
    raw_url = url_left + str(region_number) + url_right
    
    dict_for_df = {"Vacancy_names":[], "Wages_list":[], "Employers_list":[]}
    
    for i in range(0,number_of_pages):
        url = raw_url + str(i)
        driver.get(url)
        vacancy_names = driver.find_elements_by_xpath('//a[@class="serp-item__title"]')
        wages = driver.find_elements_by_xpath('//span[@class="bloko-header-section-2"]')
        employers = driver.find_elements_by_xpath('//a[@class="bloko-link bloko-link_kind-tertiary"]')
        
        print("Diagnosis info")
        print("Vacancy_names_len", len(vacancy_names))
        print("Wages_list_len", len(wages))
        print("Employers_list_len", len(employers))
        print("__________________________________")
        
        
        for p in range(len(vacancy_names)):
            dict_for_df["Vacancy_names"].append(vacancy_names[p].text)
            print("Vacancy_name: ", vacancy_names[p].text)
            
            try:
                dict_for_df["Wages_list"].append(wages[p].text)
                print("Wage: ", wages[p].text)
            except:
                dict_for_df["Wages_list"].append("-")
                print("Wage: ", "-")
                
            try:
                dict_for_df["Employers_list"].append(employers[p].text)
                print("Employer: ", employers[p].text)
            except:
                dict_for_df["Employers_list"].append("-")
                print("Employer: ", "-")

            print("__________________________________")
    
    return pd.DataFrame(dict_for_df)

In [15]:
def ParseCityAccountantVacancies(number_of_pages):
    dict_for_df = {"Vacancy_names":[], "Wages_list":[], "Employers_list":[], "Regions_list":[]}
    
    for i in range(0,number_of_pages):
        url = city_5_url + str(i)
        driver.get(url)
        
        vacancy_names = driver.find_elements_by_xpath('//a[@class="serp-item__title"]')
        wages = driver.find_elements_by_xpath('//span[@class="bloko-header-section-2"]')
        employers = driver.find_elements_by_xpath('//a[@class="bloko-link bloko-link_kind-tertiary"]')
        regions = driver.find_elements_by_xpath('//div[@class="bloko-text"]')
        
        print("Diagnosis info")
        print("Vacancy_names_len", len(vacancy_names))
        print("Wages_list_len", len(wages))
        print("Employers_list_len", len(employers))
        print("Regions_list_len", len(regions))
        print("__________________________________")
        
        
        for p in range(len(vacancy_names)):
            dict_for_df["Vacancy_names"].append(vacancy_names[p].text)
            print("Vacancy_name: ", vacancy_names[p].text)
            
            try:
                dict_for_df["Wages_list"].append(wages[p].text)
                print("Wage: ", wages[p].text)
            except:
                dict_for_df["Wages_list"].append("-")
                print("Wage: ", "-")
                
            try:
                dict_for_df["Employers_list"].append(employers[p].text)
                print("Employer: ", employers[p].text)
            except:
                dict_for_df["Employers_list"].append("-")
                print("Employer: ", "-")
                
            try:
                dict_for_df["Regions_list"].append(regions[p].text)
                print("Region: ", regions[p].text)
            except:
                dict_for_df["Regions_list"].append("-")
                print("Region: ", "-")

            print("__________________________________")
    
    return pd.DataFrame(dict_for_df)

['Заместитель главного бухгалтера',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Бухгалтер/Помощник бухгалтера',
 'Бухгалтер-казначей',
 'Бухгалтер-аналитик',
 'Главный бухгалтер',
 'Главный бухгалтер',
 'Бухгалтер-ревизор',
 'Бухгалтер-калькулятор',
 'Главный бухгалтер',
 'Бухгалтер-расчетчик',
 'Бухгалтер-кассир',
 'Главный бухгалтер',
 'Бухгалтер по учету реализации',
 'Бухгалтер',
 'Помощник главного бухгалтера',
 'Бухгалтер',
 'Бухгалтер по учету ТМЦ',
 'Специалист сопроводительной документации (Бухгалтер первичной документации)',
 'Бухгалтер-калькулятор',
 'Главный бухгалтер на УСН',
 'Помощник главного бухгалтера (внутренний туризм)',
 'Бухгалтер-кассир',
 'Главный бухгалтер',
 'Главный бухгалтер',
 'Бухгалтер',
 'Бухгалтер',
 'Экономист\\Бухгалтер по расчету заработной платы',
 'Бухгалтер по работе с региональными торговыми представителями',
 'Младший системный аналити